# Aprendizaje por refuerzo. 
*Vamos a describir el "problema del taxi". Queremos construir un taxi autónomo que pueda recoger pasajeros en uno de un conjunto de ubicaciones fijas, dejarlos en otro lugar y llegar allí en el menor tiempo posible evitando obstáculos. El AI GYM nos permite crear este entorno rápidamente*

In [3]:
import gym 
import random

random.seed(1234)

calles=gym.make("Taxi-v3").env
calles.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



*Analicemos lo que estamos viendo aquí: R, G, B e Y son lugares de recogida de entrega. La letra azul indica de dónde tenemos que recoger a alguien. La letra magenta indica a dónde quiere ir ese pasajero. Las líneas continuas representan paredes que el taxi no puede cruzar. El rectángulo lleno representa el taxi en sí: es amarillo cuando está vacío y verde cuando lleva un pasajero.*
*Nuestro pequeño mundo aquí, que hemos llamado "calles", es una cuadrícula de 5x5. El estado de este mundo en cualquier momento puede ser definido por: Dónde está el taxi (una de 5x5=25 ubicaciones). Cuál es el destino actual (4 posibilidades) Dónde está el pasajero (5 posibilidades: en uno de los destinos, o dentro del taxi)*
*Así que hay un total de 25x4x5=500 estados posibles que describen nuestro mundo. Para cada estado, hay seis acciones posibles: Move South, Nort, East, West. Recoger a un pasajero. Dejar a un transeúnte*
*Q-learning se llevará a cabo utilizando las siguientes recompensas y sanciones en cada estado: Una caída exitosa produce +20 puntos. Cada paso dado mientras conduce, un pasajero produce una penalización de -1 punto. Recoger o dejar en un lugar ilegal produce una penalización de -10 puntos. Moverse a través de una pared simplemente no está permitido en absoluto. Definamos un estado inicial, con el taxi en la ubicación (2,3), el pasajero en la ubicación de recogida 2 y el destino en la ubicación 0*


In [4]:
parada_inicial= calles.encode(2,3,2,0)
calles.s=parada_inicial
calles.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



*Examinemos la tabla de recompensas para este estado inicial*

In [5]:
calles.P[parada_inicial]

{0: [(1.0, 368, -1, False)],
 1: [(1.0, 168, -1, False)],
 2: [(1.0, 288, -1, False)],
 3: [(1.0, 248, -1, False)],
 4: [(1.0, 268, -10, False)],
 5: [(1.0, 268, -10, False)]}

*Así es como interpretan esto: cada fila corresponde a una acción potencial en este estado: moverse hacia el sur, norte, este u oeste, entrega de operaciones de recogida. Los cuatro valores de cada fila son la probabilidad asignada a esa acción, el siguiente estado que resulta de esa acción, la recompensa por esa acción y si esa acción indica que se produjo una caída exitosa. Entonces, por ejemplo, mudarnos al norte desde este estado nos pondría en el estado número 368, incurriría en una multa de -1 por tomar tiempo y no resultaría en una caída exitosa. Entonces, hagamos Q-learning primero necesitamos entrenar nuestro modelo. A un alto nivel, entrenaremos más de 10000 carreras de taxis simuladas. Para cada carrera, pasaremos el tiempo, con un 10% de probabilidad en cada paso de hacer un paso aleatorio y exploratorio en lugar de usar los valores Q aprendidos para guiar nuestras acciones.*


In [12]:
import numpy as np

q_table=np.zeros([calles.observation_space.n, calles.action_space.n])

radioAprend=0.1
factorDisc=0.6
exploracion=0.1
epocas=10000

for taxi_run in range(epocas):
    estado=calles.reset()
    term=False
    
    while not term:
        random_value=random.uniform(0,1)
        if (random_value < exploracion):
            accion=calles.action_space.sample()
        else:
            accion=np.argmax(q_table[estado])
            
        sigEstado,reward,term, info=calles.step(accion)
        
        prev_q=q_table[estado,accion]
        next_max_q=np.max(q_table[sigEstado])
        new_q=(1- radioAprend) * prev_q + radioAprend * (reward + factorDisc * next_max_q)
        q_table[estado,accion]=new_q
        
        estado=sigEstado
        

*¡así que ahora tenemos una tabla de valores Q que se pueden usar rápidamente para determinar el siguiente paso óptimo para cualquier estado dado! Revisemos la tabla para nuestro estado inicial anterior*

In [14]:
q_table[parada_inicial]

array([-2.41149071, -2.40588899, -2.40643816, -2.3639511 , -7.72682127,
       -6.30033528])

*El valor q más bajo aquí corresponde a la acción "ir al oeste", que tiene sentido, esa es la ruta más directa hacia nuestro destino desde ese punto. ¡Parece funcionar! Veámoslo en acción*

In [17]:
from IPython.display import clear_output
from time import sleep

for tripnum in range (1,3):
    estado=calles.reset()
    term=False
    
    while not term:
        accion=np.argmax(q_table[estado])
        sigEstado,reward,term, info=calles.step(accion)
        
        clear_output(wait=True)
        
        print("Trip Numero"+str(tripnum))
        print(calles.render(mode='ansi'))
        sleep(.5)
        estado=sigEstado
    sleep(2)

Trip Numero2
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

